In [1]:
import os
import numpy as np
import subprocess as sp
import pickle as pkl
import pandas as pd
import shutil

In [13]:
def Build_ET_Time_Value_Table(mapping_pkl_path,rz_node_order_file_path,coupled_mod_hgs_dir,coupled_mod_dssat_dir):
    """Build the daily ET Time Value tables, to bring DSSAT evaporation outputs into HGS

    Parameters:
    mapping_pkl_path (str): path to pickle file containing mapping information
    rz_node_order_file_path (str): path to file containing the order of HGS nodes in the root zone
    coupled_mod_hgs_dir (str): path to coupled model HGS subdirectory
    coupled_mod_dssat_dir (str): path to coupled model DSSAT subdirectory
        
            
    Returns:

   """
    # Load node list
    with open(rz_node_order_file_path,'r') as file:
        lines = file.readlines()
    nodes = [int(x.strip()) for x in lines]
    # Load mapping pickle
    with open(mapping_pkl_path,'rb') as file:
        map_dict = pkl.load(file)
    for day in range(275):
        # Blank list for vals
        vals = []
        # Load CSV's
        # Identify path to DSSAT Surface ET file
        surface_data_path = os.path.join(coupled_mod_dssat_dir,'Full_SurfaceET.txt')
        # Load
        surface_data = pd.read_csv(surface_data_path, sep = '\s+')['EOAA'].values
        # Identify path to DSSAT Soil ET file
        soil_data_path = os.path.join(coupled_mod_dssat_dir,'Full_SoilET.txt')
        # Load
        soil_data = pd.read_csv(soil_data_path, sep = '\s+')
        # Iterate through nodes
        for node in nodes:
            # Get DSSAT location information
            dssat_model,sheet,area_stat = map_dict[node]
            # Get surface ET and half of soil layer 1 for sheet 0
            if sheet == 0:
                # Grab val up from surface file
                valup = surface_data[day]
                # Grab val down from soil file layer 1
                valdn = soil_data['ES{}D'.format(sheet + 1)].values[day]
                # Set value to full surface ET + 1/2 of layer 1 ET
                val = (valup + (0.5*valdn)) * -1 * 24. * 60. / 1000. * (1./area_stat)
            # For bottom node sheet, just get half of last DSSAT layer
            elif sheet == 10:
                # Grab val down from soil file layer 1
                valup = soil_data['ES{}D'.format(sheet)].values[0]
                # Set value to 1/2 of layer 10 ET
                val = ((0.5*valup)) * -1 * 24. * 60. / 1000. * (1./area_stat)
            # For all other sheets, take half of layer above and half of layer below
            else:
                # Grab val down from soil file layer 1
                valup = soil_data['ES{}D'.format(sheet)].values[0]
                # Grab val down from soil file layer 1
                valdn = soil_data['ES{}D'.format(sheet + 1)].values[0]
                # Set value to 1/2 of layer n ET and 1/2 of layer n+1 ET
                val = ((0.5*valdn) + (0.5*valup)) * -1 * 24. * 60. / 1000. * (1./area_stat)
            # Convert DSSAT total mm to HGS total m3 and then multiply by minutes in a day to force all to be taken out in first minute of day
            vals.append(val)
        # Write out nflux.txt file
        nflux_path = os.path.join(coupled_mod_hgs_dir,'n{}flux.txt'.format(day))
        with open(nflux_path,'w') as file:
            file.write(str(len(vals))+'\n')
            lines = []
            for val in vals:
                lines.append(str(val)+'\n')
            lines[-1] = lines[-1][:-1]
            for line in lines:
                file.write(line)

In [14]:
# Establish params
mapping_pkl_path = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\mapping\lys_node_mapping.p'
rz_node_order_file_path = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\hgs\rz_node_order.txt'
coupled_mod_hgs_dir = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\hgs'
coupled_mod_dssat_dir = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\data_dssat'

In [15]:
Build_ET_Time_Value_Table(mapping_pkl_path,rz_node_order_file_path,coupled_mod_hgs_dir,coupled_mod_dssat_dir)